# <font color="red">Visibility Prediction with Random Forest Regressor</font>

# Library Import

In [1]:
# Data manipulation and visualization
import pandas as pd
import math
import json

# Preprocessing
from sklearn.model_selection import StratifiedShuffleSplit, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

import sys
# sys.path.append('../../') # Uncomment this line if running locally
sys.path.append('/kaggle/input/weatherdata') # Uncomment this line if running on Kaggle
from historyManagement import *

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Datasets Import

In [2]:
# Load the weather dataset for visibility prediction
# weather_data = pd.read_csv('../../FinalDatasets/finalDataset.csv') # Uncomment this line if running locally
weather_data = pd.read_csv('/kaggle/input/weatherdata/finalDataset.csv') # Uncomment this line if running on Kaggle
weather_data['DateTime'] = pd.to_datetime(weather_data['DateTime'])
weather_data.head()

,DateTime,Year,Month,Day,Hour,DaySegments,DaySegments_Afternoon,DaySegments_Early Morning,DaySegments_Evening,DaySegments_Late Night,...,WindDir,SeaLevelPressure,CloudCover,UVIndex,SevereRisk,Visibility,SolarRadiation,SolarEnergy,Conditions,Icon
0,2023-01-01 00:00:00,2023,1,1,0,Late Night,0,0,0,1,...,0.49,1018.68,0.03,0.0,10.0,2.76,0.0,0.0,Clear,clear-night
1,2023-01-01 01:00:00,2023,1,1,1,Late Night,0,0,0,1,...,0.54,1018.03,0.11,0.0,10.0,1.75,0.0,0.0,Clear,clear-night
2,2023-01-01 02:00:00,2023,1,1,2,Late Night,0,0,0,1,...,30.51,1017.56,0.03,0.0,10.0,1.75,0.0,0.0,Clear,clear-night
3,2023-01-01 03:00:00,2023,1,1,3,Late Night,0,0,0,1,...,49.23,1018.05,0.00,0.0,10.0,2.28,0.0,0.0,Clear,clear-night
4,2023-01-01 04:00:00,2023,1,1,4,Late Night,0,0,0,1,...,49.90,1018.00,86.17,0.0,10.0,1.27,0.0,0.0,Partially cloudy,fog


In [3]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateTime                   14616 non-null  datetime64[ns]
 1   Year                       14616 non-null  int64         
 2   Month                      14616 non-null  int64         
 3   Day                        14616 non-null  int64         
 4   Hour                       14616 non-null  int64         
 5   DaySegments                14616 non-null  object        
 6   DaySegments_Afternoon      14616 non-null  int64         
 7   DaySegments_Early Morning  14616 non-null  int64         
 8   DaySegments_Evening        14616 non-null  int64         
 9   DaySegments_Late Night     14616 non-null  int64         
 10  DaySegments_Midday         14616 non-null  int64         
 11  DaySegments_Morning        14616 non-null  int64         
 12  DayS

In [4]:
weather_data.describe()

,DateTime,Year,Month,Day,Hour,DaySegments_Afternoon,DaySegments_Early Morning,DaySegments_Evening,DaySegments_Late Night,DaySegments_Midday,...,Windgust,WindSpeed,WindDir,SeaLevelPressure,CloudCover,UVIndex,SevereRisk,Visibility,SolarRadiation,SolarEnergy
count,14616,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,...,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000
mean,2023-11-01 11:30:00,2023.400657,5.720854,15.735632,11.500000,0.131294,0.060550,0.098043,0.249726,0.116585,...,17.068560,8.090783,159.739178,1008.265012,53.579923,2.263555,16.504787,4.368134,227.640683,0.819217
min,2023-01-01 00:00:00,2023.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.660000,0.000000,0.000000,982.420000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,2023-06-02 05:45:00,2023.000000,3.000000,8.000000,5.750000,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.170000,2.870000,80.965000,1003.277500,26.670000,0.000000,10.000000,3.990000,0.000000,0.000000
50%,2023-11-01 11:30:00,2023.000000,6.000000,16.000000,11.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,14.645000,7.670000,163.790000,1007.970000,51.540000,0.000000,10.000000,4.650000,10.665000,0.010000
75%,2024-04-01 17:15:00,2024.000000,8.000000,23.000000,17.250000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.410000,11.750000,241.402500,1013.400000,87.790000,4.740000,12.580000,4.650000,459.745000,1.660000
max,2024-08-31 23:00:00,2024.000000,12.000000,31.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,84.120000,71.970000,360.000000,1022.060000,100.000000,10.000000,97.740000,24.100000,1026.650000,3.700000
std,NaN,0.490048,3.195073,8.803610,6.922423,0.337734,0.238511,0.297383,0.432869,0.320936,...,10.195911,6.250203,104.422027,6.010683,35.166637,3.083074,15.031827,1.376994,307.219387,1.106444


# Data Split

In [5]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
for train_idx, test_idx in sss.split(weather_data, weather_data['DaySegments']):
    weather_data_train = weather_data.iloc[train_idx]
    weather_data_test = weather_data.iloc[test_idx]

In [6]:
# Show the count of each unique class in DaySegments
class_counts = weather_data['DaySegments'].value_counts().reset_index()
class_counts.columns = ['DaySegments', 'Count']

print("Distribution of data based on DaySegments:")
print(class_counts)

Distribution of data based on DaySegments:
     DaySegments  Count
0     Late Night   3650
1        Morning   3504
2      Afternoon   1919
3         Midday   1704
4          Night   1521
5        Evening   1433
6  Early Morning    885


In [7]:
print("Distribution of DaySegments in Training Set:")
print(weather_data_train['DaySegments'].value_counts())

print("\nDistribution of DaySegments in Test Set:")
print(weather_data_test['DaySegments'].value_counts())

Distribution of DaySegments in Training Set:
DaySegments
Late Night       2920
Morning          2803
Afternoon        1535
Midday           1363
Night            1217
Evening          1146
Early Morning     708
Name: count, dtype: int64

Distribution of DaySegments in Test Set:
DaySegments
Late Night       730
Morning          701
Afternoon        384
Midday           341
Night            304
Evening          287
Early Morning    177
Name: count, dtype: int64


# Some Reused Parameters

In [8]:
# For bootstrap=True cases
bootstrap_true_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True],
    'max_samples': [None, 0.8, 0.9],
    'min_impurity_decrease': [0.0, 0.01, 0.02]
}

# For bootstrap=False cases  
bootstrap_false_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [False],
    'min_impurity_decrease': [0.0, 0.01, 0.02]
}

grid_param = [bootstrap_true_params, bootstrap_false_params]

In [9]:
number_of_model_for_randomized_grid = 600 # Estimated run time 10 hours
model_name_for_saving = "Random Forest"
target_name_for_saving = "Visibility"

# All Features

In [10]:
X_train_raw = weather_data_train.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon']) # Target variable
y_train = weather_data_train['Visibility']

In [11]:
X_test_raw = weather_data_test.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon']) # Target variable
y_test = weather_data_test['Visibility']

In [12]:
feature_columns = [col for col in X_train_raw.columns]
feature_columns_for_saving = ','.join(feature_columns)

In [13]:
print("X_train: ", len(X_train_raw))
print("y_train: ", len(y_train))
print("\nX_test: ", len(X_test_raw))
print("y_test: ", len(y_test))

X_train:  11692
y_train:  11692

X_test:  2924
y_test:  2924


## MinMax Scaler

In [14]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [15]:
X_train

array([[0.72727273, 0.83333333, 0.43478261, ..., 0.4922    , 0.697     ,
        0.31560059],
       [0.90909091, 0.26666667, 0.69565217, ..., 0.        , 0.303     ,
        0.07388643],
       [0.09090909, 0.43333333, 0.        , ..., 0.        , 0.        ,
        0.07388643],
       ...,
       [0.63636364, 0.86666667, 0.43478261, ..., 0.8756    , 0.306     ,
        0.09436352],
       [0.        , 0.4       , 0.91304348, ..., 0.        , 0.        ,
        0.07388643],
       [0.72727273, 0.26666667, 0.34782609, ..., 0.8925    , 0.29      ,
        0.27812962]])

In [16]:
X_test

array([[0.63636364, 0.06666667, 0.95652174, ..., 0.894     , 0.        ,
        0.07388643],
       [0.45454545, 0.26666667, 0.65217391, ..., 0.9677    , 0.074     ,
        0.07388643],
       [0.63636364, 0.03333333, 0.2173913 , ..., 0.9935    , 0.        ,
        0.07388643],
       ...,
       [0.09090909, 0.2       , 0.73913043, ..., 0.0013    , 0.203     ,
        0.07388643],
       [0.27272727, 0.13333333, 0.86956522, ..., 0.        , 0.        ,
        0.25575259],
       [0.45454545, 0.73333333, 0.95652174, ..., 0.5161    , 0.        ,
        0.07388643]])

### Default Parameters

In [17]:
training_model = RandomForestRegressor(random_state=42)
training_model

RandomForestRegressor(random_state=42)

In [18]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 14.3 s, sys: 56.1 ms, total: 14.3 s
Wall time: 14.3 s


RandomForestRegressor(random_state=42)

In [19]:
y_pred = training_model.predict(X_test)
y_pred

array([4.65  , 3.8406, 4.3719, ..., 3.7515, 4.713 , 4.6552])

In [20]:
y_test

5158     4.65
3831     3.70
5117     4.19
6326     3.88
11359    3.68
         ... 
10290    5.17
13995    4.48
9665     3.91
2276     4.00
4174     4.65
Name: Visibility, Length: 2924, dtype: float64

In [21]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [22]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.8680991684131328
MAE Score: 0.3667290697674418
R2 Score: 0.5551335391215393
RMSE Score: 0.9317183954463564


In [23]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.868099, R²: 0.555134
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Randomized Grid Search

In [24]:
grid_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [25]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 600 candidates, totalling 3000 fits
CPU times: user 37.1 s, sys: 10.6 s, total: 47.7 s
Wall time: 1h 41min 55s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=600, n_jobs=-1,
                   param_distributions=[{'bootstrap': [True],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'max_samples': [None, 0.8, 0.9],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]},
                                        {'bootstrap': [False],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]}],
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [26]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0.0, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
Best Cross-Validation Score (Negative MSE): -0.6726149589500539
Best Cross-Validation Score (MSE): 0.6726149589500539
Best Cross-Validation Score (RMSE): 0.82013106205658


In [27]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [28]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [29]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.7359115099433431
MAE Score: 0.3501060989512084
R2 Score: 0.622874481556448
RMSE Score: 0.8578528486537438


In [30]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.735912, R²: 0.622874
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


## Standard Scaler

In [31]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [32]:
X_train

array([[ 1.01884116,  1.17229173, -0.21938027, ..., -0.12088825,
         1.52387472,  1.09644529],
       [ 1.64376461, -0.75947244,  0.64624572, ..., -1.52028666,
         0.24645772, -0.43343649],
       [-1.16839092, -0.1913065 , -1.66209024, ..., -1.52028666,
        -0.73592134, -0.43343649],
       ...,
       [ 0.70637943,  1.28592492, -0.21938027, ...,  0.96917545,
         0.25618424, -0.30383078],
       [-1.48085264, -0.30493969,  1.36760071, ..., -1.52028666,
        -0.73592134, -0.43343649],
       [ 1.01884116, -0.75947244, -0.50792226, ...,  1.01722469,
         0.20430944,  0.85928021]])

In [33]:
X_test

array([[ 0.70637943, -1.44127155,  1.51187171, ...,  1.02148941,
        -0.73592134, -0.43343649],
       [ 0.08145598, -0.75947244,  0.50197472, ...,  1.23102956,
        -0.49600038, -0.43343649],
       [ 0.70637943, -1.55490474, -0.94073525, ...,  1.30438283,
        -0.73592134, -0.43343649],
       ...,
       [-1.16839092, -0.98673881,  0.79051672, ..., -1.51659057,
        -0.07775979, -0.43343649],
       [-0.54346747, -1.21400518,  1.22332971, ..., -1.52028666,
        -0.73592134,  0.71764924],
       [ 0.08145598,  0.83139217,  1.51187171, ..., -0.05293696,
        -0.73592134, -0.43343649]])

### Default Parameters

In [34]:
training_model = RandomForestRegressor(random_state=42)
training_model

RandomForestRegressor(random_state=42)

In [35]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 14.3 s, sys: 0 ns, total: 14.3 s
Wall time: 14.3 s


RandomForestRegressor(random_state=42)

In [36]:
y_pred = training_model.predict(X_test)
y_pred

array([4.65  , 3.838 , 4.3676, ..., 3.7608, 4.7088, 4.6552])

In [37]:
y_test

5158     4.65
3831     3.70
5117     4.19
6326     3.88
11359    3.68
         ... 
10290    5.17
13995    4.48
9665     3.91
2276     4.00
4174     4.65
Name: Visibility, Length: 2924, dtype: float64

In [38]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [39]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.8637863868228457
MAE Score: 0.3657951094391244
R2 Score: 0.5573436689689392
RMSE Score: 0.9294010903925418


In [40]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.863786, R²: 0.557344
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Randomized Grid Search

In [41]:
grid_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [42]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 600 candidates, totalling 3000 fits
CPU times: user 36.6 s, sys: 10.5 s, total: 47.1 s
Wall time: 1h 41min 57s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=600, n_jobs=-1,
                   param_distributions=[{'bootstrap': [True],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'max_samples': [None, 0.8, 0.9],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]},
                                        {'bootstrap': [False],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]}],
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [43]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0.0, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
Best Cross-Validation Score (Negative MSE): -0.6731792422824221
Best Cross-Validation Score (MSE): 0.6731792422824221
Best Cross-Validation Score (RMSE): 0.8204750101510845


In [44]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [45]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [46]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.7359677768863361
MAE Score: 0.34995220398236815
R2 Score: 0.6228456469754409
RMSE Score: 0.8578856432452615


In [47]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.735968, R²: 0.622846
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


## Robust Scaler

In [48]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [49]:
X_train

array([[ 0.6       ,  0.66666667, -0.16666667, ..., -0.03738547,
         1.44008264,  8.87596899],
       [ 1.        , -0.46666667,  0.33333333, ..., -0.84268652,
         0.62603306,  0.        ],
       [-0.8       , -0.13333333, -1.        , ..., -0.84268652,
         0.        ,  0.        ],
       ...,
       [ 0.4       ,  0.73333333, -0.16666667, ...,  0.5899051 ,
         0.6322314 ,  0.75193798],
       [-1.        , -0.2       ,  0.75      , ..., -0.84268652,
         0.        ,  0.        ],
       [ 0.6       , -0.46666667, -0.33333333, ...,  0.61755563,
         0.59917355,  7.5       ]])

In [50]:
X_test

array([[ 4.00000000e-01, -8.66666667e-01,  8.33333333e-01, ...,
         6.20009817e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -4.66666667e-01,  2.50000000e-01, ...,
         7.40592277e-01,  1.52892562e-01,  0.00000000e+00],
       [ 4.00000000e-01, -9.33333333e-01, -5.83333333e-01, ...,
         7.82804319e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000000e-01, -6.00000000e-01,  4.16666667e-01, ...,
        -8.40559555e-01,  4.19421488e-01,  0.00000000e+00],
       [-4.00000000e-01, -7.33333333e-01,  6.66666667e-01, ...,
        -8.42686518e-01,  0.00000000e+00,  6.67829457e+00],
       [ 0.00000000e+00,  4.66666667e-01,  8.33333333e-01, ...,
         1.71793194e-03,  0.00000000e+00,  0.00000000e+00]])

### Default Parameters

In [51]:
training_model = RandomForestRegressor(random_state=42)
training_model

RandomForestRegressor(random_state=42)

In [52]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 14.4 s, sys: 0 ns, total: 14.4 s
Wall time: 14.4 s


RandomForestRegressor(random_state=42)

In [53]:
y_pred = training_model.predict(X_test)
y_pred

array([4.65  , 3.8406, 4.3717, ..., 3.7515, 4.7088, 4.6552])

In [54]:
y_test

5158     4.65
3831     3.70
5117     4.19
6326     3.88
11359    3.68
         ... 
10290    5.17
13995    4.48
9665     3.91
2276     4.00
4174     4.65
Name: Visibility, Length: 2924, dtype: float64

In [55]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [56]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.8688912617339263
MAE Score: 0.366941073871409
R2 Score: 0.5547276226489422
RMSE Score: 0.9321433697312481


In [57]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.868891, R²: 0.554728
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Randomized Grid Search

In [58]:
grid_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [59]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 600 candidates, totalling 3000 fits
CPU times: user 37 s, sys: 10.7 s, total: 47.7 s
Wall time: 1h 42min 8s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=600, n_jobs=-1,
                   param_distributions=[{'bootstrap': [True],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'max_samples': [None, 0.8, 0.9],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]},
                                        {'bootstrap': [False],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]}],
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [60]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0.0, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
Best Cross-Validation Score (Negative MSE): -0.6727568474047827
Best Cross-Validation Score (MSE): 0.6727568474047827
Best Cross-Validation Score (RMSE): 0.8202175610195034


In [61]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [62]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [63]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.7360774168118144
MAE Score: 0.35004837931296545
R2 Score: 0.6227894608536322
RMSE Score: 0.8579495421129464


In [64]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.736077, R²: 0.622789
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


# Selected Features

In [65]:
numeric_cols = weather_data.select_dtypes(include=[float, int]).columns
correlations = weather_data[numeric_cols].corr()['Visibility'].sort_values(ascending=False)
print(correlations)

Visibility                   1.000000
Temp                         0.445302
FeelsLike                    0.420084
Dew                          0.263350
Season_Rainy                 0.205205
Windgust                     0.177655
Month                        0.170496
WindSpeed                    0.160971
SevereRisk                   0.139495
Season_Summer                0.111505
Hour                         0.108095
Year                         0.094151
DaySegments_Evening          0.093931
DaySegments_Midday           0.081819
CloudCover                   0.078414
PrecipProb                   0.077142
Day                          0.067361
UVIndex                      0.055563
SolarEnergy                  0.054855
SolarRadiation               0.054367
DaySegments_Night            0.052737
DaySegments_Afternoon        0.042864
Season_Autumn                0.028047
WindDir                     -0.003214
Precip                      -0.003620
DaySegments_Late Night      -0.010289
DaySegments_

In [66]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateTime                   14616 non-null  datetime64[ns]
 1   Year                       14616 non-null  int64         
 2   Month                      14616 non-null  int64         
 3   Day                        14616 non-null  int64         
 4   Hour                       14616 non-null  int64         
 5   DaySegments                14616 non-null  object        
 6   DaySegments_Afternoon      14616 non-null  int64         
 7   DaySegments_Early Morning  14616 non-null  int64         
 8   DaySegments_Evening        14616 non-null  int64         
 9   DaySegments_Late Night     14616 non-null  int64         
 10  DaySegments_Midday         14616 non-null  int64         
 11  DaySegments_Morning        14616 non-null  int64         
 12  DayS

## Run Again

In [67]:
X_train_raw = weather_data_train.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon', # Target variable
                                                
                                                'Day', # same day from different month act differently
                                                'WindDir', # very low correlation with Visibility
                                                'Precip']) # very low correlation with Visibility
y_train = weather_data_train['Visibility']

In [68]:
X_test_raw = weather_data_test.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon', # Target variable
                                                
                                                'Day', # same day from different month act differently
                                                'WindDir', # very low correlation with Visibility
                                                'Precip']) # very low correlation with Visibility
y_test = weather_data_test['Visibility']

In [69]:
feature_columns = [col for col in X_train_raw.columns]
feature_columns_for_saving = ','.join(feature_columns)

In [70]:
print("X_train: ", len(X_train_raw))
print("y_train: ", len(y_train))
print("\nX_test: ", len(X_test_raw))
print("y_test: ", len(y_test))

X_train:  11692
y_train:  11692

X_test:  2924
y_test:  2924


### MinMax Scaler

In [71]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [72]:
X_train

array([[0.72727273, 0.43478261, 0.        , ..., 0.4922    , 0.697     ,
        0.31560059],
       [0.90909091, 0.69565217, 1.        , ..., 0.        , 0.303     ,
        0.07388643],
       [0.09090909, 0.        , 0.        , ..., 0.        , 0.        ,
        0.07388643],
       ...,
       [0.63636364, 0.43478261, 0.        , ..., 0.8756    , 0.306     ,
        0.09436352],
       [0.        , 0.91304348, 0.        , ..., 0.        , 0.        ,
        0.07388643],
       [0.72727273, 0.34782609, 0.        , ..., 0.8925    , 0.29      ,
        0.27812962]])

In [73]:
X_test

array([[0.63636364, 0.95652174, 0.        , ..., 0.894     , 0.        ,
        0.07388643],
       [0.45454545, 0.65217391, 1.        , ..., 0.9677    , 0.074     ,
        0.07388643],
       [0.63636364, 0.2173913 , 0.        , ..., 0.9935    , 0.        ,
        0.07388643],
       ...,
       [0.09090909, 0.73913043, 1.        , ..., 0.0013    , 0.203     ,
        0.07388643],
       [0.27272727, 0.86956522, 0.        , ..., 0.        , 0.        ,
        0.25575259],
       [0.45454545, 0.95652174, 0.        , ..., 0.5161    , 0.        ,
        0.07388643]])

#### Default Parameters

In [74]:
training_model = RandomForestRegressor(random_state=42)
training_model

RandomForestRegressor(random_state=42)

In [75]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 11.9 s, sys: 0 ns, total: 11.9 s
Wall time: 12 s


RandomForestRegressor(random_state=42)

In [76]:
y_pred = training_model.predict(X_test)
y_pred

array([4.6341, 3.7109, 4.2644, ..., 3.6337, 4.579 , 4.6604])

In [77]:
y_test

5158     4.65
3831     3.70
5117     4.19
6326     3.88
11359    3.68
         ... 
10290    5.17
13995    4.48
9665     3.91
2276     4.00
4174     4.65
Name: Visibility, Length: 2924, dtype: float64

In [78]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [79]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 1.0080022193741454
MAE Score: 0.37901111491108064
R2 Score: 0.4834387634418268
RMSE Score: 1.0039931371150628


In [80]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

❌ Regression model not improved:
   Current MSE: 1.008002, Best MSE: 0.868099
   Current R²: 0.483439, Best R²: 0.555134
   Current Features: 24, Best Features: 27


#### Tuning with Randomized Grid Search

In [81]:
grid_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [82]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 600 candidates, totalling 3000 fits
CPU times: user 31 s, sys: 9.38 s, total: 40.4 s
Wall time: 1h 23min 35s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=600, n_jobs=-1,
                   param_distributions=[{'bootstrap': [True],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'max_samples': [None, 0.8, 0.9],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]},
                                        {'bootstrap': [False],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]}],
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [83]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0.0, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
Best Cross-Validation Score (Negative MSE): -0.965167401862417
Best Cross-Validation Score (MSE): 0.965167401862417
Best Cross-Validation Score (RMSE): 0.9824293368290755


In [84]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [85]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [86]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.8511340320317967
MAE Score: 0.36641692126463027
R2 Score: 0.5638274999671437
RMSE Score: 0.9225692559541515


In [87]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

❌ Regression model not improved:
   Current MSE: 0.851134, Best MSE: 0.735912
   Current R²: 0.563827, Best R²: 0.622874
   Current Features: 24, Best Features: 27


### Standard Scaler

In [88]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [89]:
X_train

array([[ 1.01884116, -0.21938027, -0.38875095, ..., -0.12088825,
         1.52387472,  1.09644529],
       [ 1.64376461,  0.64624572,  2.57234098, ..., -1.52028666,
         0.24645772, -0.43343649],
       [-1.16839092, -1.66209024, -0.38875095, ..., -1.52028666,
        -0.73592134, -0.43343649],
       ...,
       [ 0.70637943, -0.21938027, -0.38875095, ...,  0.96917545,
         0.25618424, -0.30383078],
       [-1.48085264,  1.36760071, -0.38875095, ..., -1.52028666,
        -0.73592134, -0.43343649],
       [ 1.01884116, -0.50792226, -0.38875095, ...,  1.01722469,
         0.20430944,  0.85928021]])

In [90]:
X_test

array([[ 0.70637943,  1.51187171, -0.38875095, ...,  1.02148941,
        -0.73592134, -0.43343649],
       [ 0.08145598,  0.50197472,  2.57234098, ...,  1.23102956,
        -0.49600038, -0.43343649],
       [ 0.70637943, -0.94073525, -0.38875095, ...,  1.30438283,
        -0.73592134, -0.43343649],
       ...,
       [-1.16839092,  0.79051672,  2.57234098, ..., -1.51659057,
        -0.07775979, -0.43343649],
       [-0.54346747,  1.22332971, -0.38875095, ..., -1.52028666,
        -0.73592134,  0.71764924],
       [ 0.08145598,  1.51187171, -0.38875095, ..., -0.05293696,
        -0.73592134, -0.43343649]])

#### Default Parameters

In [91]:
training_model = RandomForestRegressor(random_state=42)
training_model

RandomForestRegressor(random_state=42)

In [92]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 11.8 s, sys: 0 ns, total: 11.8 s
Wall time: 11.8 s


RandomForestRegressor(random_state=42)

In [93]:
y_pred = training_model.predict(X_test)
y_pred

array([4.6314, 3.7073, 4.2644, ..., 3.6429, 4.579 , 4.6604])

In [94]:
y_test

5158     4.65
3831     3.70
5117     4.19
6326     3.88
11359    3.68
         ... 
10290    5.17
13995    4.48
9665     3.91
2276     4.00
4174     4.65
Name: Visibility, Length: 2924, dtype: float64

In [95]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [96]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 1.0080823825854996
MAE Score: 0.3792312585499315
R2 Score: 0.48339768296919716
RMSE Score: 1.004033058512268


In [97]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

❌ Regression model not improved:
   Current MSE: 1.008082, Best MSE: 0.863786
   Current R²: 0.483398, Best R²: 0.557344
   Current Features: 24, Best Features: 27


#### Tuning with Randomized Grid Search

In [98]:
grid_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [99]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 600 candidates, totalling 3000 fits
CPU times: user 30.9 s, sys: 9.59 s, total: 40.5 s
Wall time: 1h 23min 36s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=600, n_jobs=-1,
                   param_distributions=[{'bootstrap': [True],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'max_samples': [None, 0.8, 0.9],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]},
                                        {'bootstrap': [False],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]}],
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [100]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0.0, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
Best Cross-Validation Score (Negative MSE): -0.9640348000360074
Best Cross-Validation Score (MSE): 0.9640348000360074
Best Cross-Validation Score (RMSE): 0.9818527384674381


In [101]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [102]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [103]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.8527221929147116
MAE Score: 0.36683571496428063
R2 Score: 0.5630136303805854
RMSE Score: 0.923429582001092


In [104]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

❌ Regression model not improved:
   Current MSE: 0.852722, Best MSE: 0.735968
   Current R²: 0.563014, Best R²: 0.622846
   Current Features: 24, Best Features: 27


### Robust Scaler

In [105]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [106]:
X_train

array([[ 0.6       , -0.16666667,  0.        , ..., -0.03738547,
         1.44008264,  8.87596899],
       [ 1.        ,  0.33333333,  1.        , ..., -0.84268652,
         0.62603306,  0.        ],
       [-0.8       , -1.        ,  0.        , ..., -0.84268652,
         0.        ,  0.        ],
       ...,
       [ 0.4       , -0.16666667,  0.        , ...,  0.5899051 ,
         0.6322314 ,  0.75193798],
       [-1.        ,  0.75      ,  0.        , ..., -0.84268652,
         0.        ,  0.        ],
       [ 0.6       , -0.33333333,  0.        , ...,  0.61755563,
         0.59917355,  7.5       ]])

In [107]:
X_test

array([[ 4.00000000e-01,  8.33333333e-01,  0.00000000e+00, ...,
         6.20009817e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  2.50000000e-01,  1.00000000e+00, ...,
         7.40592277e-01,  1.52892562e-01,  0.00000000e+00],
       [ 4.00000000e-01, -5.83333333e-01,  0.00000000e+00, ...,
         7.82804319e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000000e-01,  4.16666667e-01,  1.00000000e+00, ...,
        -8.40559555e-01,  4.19421488e-01,  0.00000000e+00],
       [-4.00000000e-01,  6.66666667e-01,  0.00000000e+00, ...,
        -8.42686518e-01,  0.00000000e+00,  6.67829457e+00],
       [ 0.00000000e+00,  8.33333333e-01,  0.00000000e+00, ...,
         1.71793194e-03,  0.00000000e+00,  0.00000000e+00]])

#### Default Parameters

In [108]:
training_model = RandomForestRegressor(random_state=42)
training_model

RandomForestRegressor(random_state=42)

In [109]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 11.9 s, sys: 0 ns, total: 11.9 s
Wall time: 11.9 s


RandomForestRegressor(random_state=42)

In [110]:
y_pred = training_model.predict(X_test)
y_pred

array([4.6341, 3.7083, 4.2645, ..., 3.6337, 4.579 , 4.6604])

In [111]:
y_test

5158     4.65
3831     3.70
5117     4.19
6326     3.88
11359    3.68
         ... 
10290    5.17
13995    4.48
9665     3.91
2276     4.00
4174     4.65
Name: Visibility, Length: 2924, dtype: float64

In [112]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [113]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 1.0091599438030099
MAE Score: 0.37886826265389867
R2 Score: 0.4828454754995253
RMSE Score: 1.0045695315920198


In [114]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

❌ Regression model not improved:
   Current MSE: 1.009160, Best MSE: 0.868891
   Current R²: 0.482845, Best R²: 0.554728
   Current Features: 24, Best Features: 27


#### Tuning with Randomized Grid Search

In [115]:
grid_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [116]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 600 candidates, totalling 3000 fits
CPU times: user 31.4 s, sys: 9.4 s, total: 40.8 s
Wall time: 1h 23min 46s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=600, n_jobs=-1,
                   param_distributions=[{'bootstrap': [True],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'max_samples': [None, 0.8, 0.9],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]},
                                        {'bootstrap': [False],
                                         'max_depth': [None, 10, 20, 30],
                                         'max_features': ['sqrt', 'log2', None],
                                         'min_impurity_decrease': [0.0, 0.01,
                                                                   0.02],
                                         'min_samples_leaf': [1, 2, 4, 6],
                                         'min_samples_split': [2, 5, 10, 15,
                                                               20],
                                         'n_estimators': [100, 200, 300]}],
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [117]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0.0, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
Best Cross-Validation Score (Negative MSE): -0.964090078474215
Best Cross-Validation Score (MSE): 0.964090078474215
Best Cross-Validation Score (RMSE): 0.9818808881296218


In [118]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [119]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [120]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.8542002623787206
MAE Score: 0.36677758492932094
R2 Score: 0.5622561782883455
RMSE Score: 0.9242295506954539


In [121]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

❌ Regression model not improved:
   Current MSE: 0.854200, Best MSE: 0.736077
   Current R²: 0.562256, Best R²: 0.622789
   Current Features: 24, Best Features: 27


# All Performance

In [122]:
show_model_history('regression', model_name=model_name_for_saving, target_column=target_name_for_saving)


📊 REGRESSION Model Performance History
              Model         Scaler     Target  Features      MSE      MAE     RMSE       R²
Random Forest Tuned   MinMaxScaler Visibility        27 0.735912 0.350106 0.857853 0.622874
Random Forest Tuned StandardScaler Visibility        27 0.735968 0.349952 0.857886 0.622846
Random Forest Tuned   RobustScaler Visibility        27 0.736077 0.350048 0.857950 0.622789
      Random Forest StandardScaler Visibility        27 0.863786 0.365795 0.929401 0.557344
      Random Forest   MinMaxScaler Visibility        27 0.868099 0.366729 0.931718 0.555134
      Random Forest   RobustScaler Visibility        27 0.868891 0.366941 0.932143 0.554728

📈 Total models shown: 6
🏆 Best R² Score: 0.622874 (Random Forest Tuned + MinMaxScaler for Visibility)


# <center><font size="50" color="red">Thank You</font></center>